# Stacking

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import gc

print('loading files...')
train = pd.read_csv('train.csv', na_values=-1) # nrows=10000
test = pd.read_csv('test.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)

loading files...
(595212, 39) (892816, 38)


In [2]:
# custom objective function (similar to auc)

def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)

def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

def gini_lgb(preds, dtrain):
    y = list(dtrain.get_label())
    score = gini(y, preds) / gini(y, y)
    return 'gini', score, True

In [3]:
# xgb
params = {'eta': 0.02, 'max_depth': 4,
          'subsample': 0.9,
          'colsample_bytree': 0.9, 
          'objective': 'binary:logistic',
          'eval_metric': 'auc',
          'silent': True}

In [4]:
X = train.drop(['id', 'target'], axis=1)

In [5]:
features = X.columns

In [6]:
features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15'],
      dtype='object')

In [7]:
X = X.values

In [8]:
X

array([[ 2.        ,  2.        ,  5.        , ...,  0.8836789 ,
         0.3708099 ,  3.6055512 ],
       [ 1.        ,  1.        ,  7.        , ...,  0.6188165 ,
         0.38871583,  2.4494898 ],
       [ 5.        ,  4.        ,  9.        , ...,  0.6415857 ,
         0.3472751 ,  3.3166249 ],
       ...,
       [ 1.        ,  1.        , 10.        , ...,  0.5963733 ,
         0.39874804,  1.7320508 ],
       [ 5.        ,  2.        ,  3.        , ...,  0.7644341 ,
         0.38496754,  3.1622777 ],
       [ 0.        ,  1.        ,  8.        , ...,  0.9326493 ,
         0.37802115,  3.7416575 ]], dtype=float32)

In [9]:
y = train['target'].values

In [10]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [11]:
train['target']

0         0
1         0
2         0
3         0
4         0
         ..
595207    0
595208    0
595209    0
595210    0
595211    0
Name: target, Length: 595212, dtype: int64

In [12]:
sub = test['id'].to_frame()

In [13]:
sub

,id
0,0
1,1
2,2
3,3
4,4
...,...
892811,1488022
892812,1488023
892813,1488024
892814,1488025


In [14]:
sub['target'] = 0

In [15]:
sub

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
892811,1488022,0
892812,1488023,0
892813,1488024,0
892814,1488025,0


In [16]:
nrounds=2000 
kfold = 2 
skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=0)

- nrounds=200: 이 변수는 랜덤 포레스트 또는 부스팅 모델 등의 알고리즘에서 사용되는 트리의 개수를 나타내는 것으로 보입니다. nrounds는 모델이 훈련될 때 사용되는 트리의 수를 설정하는 역할을 합니다. 이 값이 크면 모델의 복잡도가 증가하게 되므로 과적합의 가능성을 고려해야 합니다.

- kfold = 2: 이 변수는 교차 검증(Cross-validation)의 폴드 수를 나타냅니다. 교차 검증은 모델의 성능을 평가하거나 하이퍼파라미터를 조정하기 위해 사용되는 방법 중 하나로, 데이터를 여러 개의 폴드로 나누고 각 폴드를 순서대로 검증 데이터로 사용하여 모델을 훈련하고 평가하는 과정입니다.

- skf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state=0): 이 코드는 Scikit-learn의 StratifiedKFold 교차 검증 분할기를 생성하는 부분입니다.

- n_splits=kfold: 교차 검증의 폴드 수를 설정합니다. 여기서는 kfold 변수에 할당된 값인 2를 사용하여 2개의 폴드로 데이터를 나누겠다는 것을 나타냅니다.
- shuffle=True: 데이터를 섞어서 분할할지 여부를 설정합니다. True로 설정하면 데이터를 섞어서 분할합니다.
- random_state=0: 난수 생성 시 사용되는 시드 값을 설정합니다. 이를 통해 같은 시드 값을 사용하면 동일한 랜덤 시퀀스가 생성되어 재현 가능한 결과를 얻을 수 있습니다.
-  이렇게 설정된 StratifiedKFold 분할기는 데이터를 교차 검증을 위해 설정한 폴드 수대로 나누고, 섞은 후에 폴드별로 모델을 훈련하고 검증하는데 사용될 것입니다.






In [17]:
for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    break

In [18]:
train_index

array([     0,      3,      4, ..., 595207, 595210, 595211])

In [19]:
valid_index

array([     1,      2,      8, ..., 595205, 595208, 595209])

In [20]:
y.mean()

0.036447517859182946

In [21]:
y[train_index].mean()

0.036447517859182946

In [22]:
X_train, X_valid = X[train_index], X[valid_index]

In [23]:
y_train, y_valid = y[train_index], y[valid_index]

In [24]:
X_train

array([[ 2.        ,  2.        ,  5.        , ...,  0.8836789 ,
         0.3708099 ,  3.6055512 ],
       [ 0.        ,  1.        ,  2.        , ...,  0.5429488 ,
         0.29495764,  2.        ],
       [ 0.        ,  2.        ,  0.        , ...,  0.5658315 ,
         0.36510274,  2.        ],
       ...,
       [ 3.        ,  1.        , 10.        , ...,  0.6846306 ,
         0.3854867 ,  2.6457512 ],
       [ 5.        ,  2.        ,  3.        , ...,  0.7644341 ,
         0.38496754,  3.1622777 ],
       [ 0.        ,  1.        ,  8.        , ...,  0.9326493 ,
         0.37802115,  3.7416575 ]], dtype=float32)

In [25]:
y_train

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [26]:
d_train = xgb.DMatrix(X_train, y_train) 
d_valid = xgb.DMatrix(X_valid, y_valid) 

- XGBoost에서 사용되는 데이터 형식인 DMatrix로 데이터를 변환하는 부분입니다. XGBoost 모델을 학습하기 위해서는 데이터를 DMatrix 형태로 변환해야 합니다. 이를 통해 XGBoost 내부에서 최적화된 데이터 구조를 사용하여 효율적인 학습과 예측을 할 수 있습니다.

- d_train = xgb.DMatrix(X_train, y_train): DMatrix를 생성하는데, 여기서 X_train은 훈련 데이터의 입력 특성(features)이고 y_train은 해당 데이터의 타겟 변수(target)입니다. 즉, X_train과 y_train을 훈련 데이터로 사용하여 DMatrix를 생성합니다. 이 데이터 형식은 XGBoost 내부의 알고리즘이 최적화된 방식으로 데이터를 처리할 수 있도록 돕습니다.

- d_valid = xgb.DMatrix(X_valid, y_valid): 마찬가지로 검증 데이터인 X_valid와 y_valid를 사용하여 검증용 DMatrix를 생성합니다. 이 데이터 형식은 모델을 학습할 때와 예측할 때 모두 필요합니다.

- DMatrix는 XGBoost 모델을 사용하기 위한 특별한 데이터 객체로, 더욱 효율적인 메모리 사용과 빠른 계산을 위해 설계되었습니다. 이를 통해 XGBoost 모델이 데이터를 더 빠르게 처리하고 복잡한 피처 엔지니어링을 수행할 수 있게 됩니다.

In [27]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
#  모델의 성능을 모니터링 : 여기서는 roc_auc 를 보다가...

- XGBoost 모델을 훈련하는 부분
- eta: 0.02: 학습 속도를 나타내는 하이퍼파라미터로, 각 트리의 가중치 갱신에 적용되는 값입니다. 작은 값이면 학습이 더 안정적으로 수행될 수 있지만 수렴하는데 더 많은 시간이 걸릴 수 있습니다.

- max_depth: 4: 각 트리의 최대 깊이를 제한하는 하이퍼파라미터입니다. 과적합을 피하기 위해 트리의 깊이를 제한할 수 있습니다.

- subsample: 0.9: 각 트리마다 사용되는 훈련 데이터의 일부를 무작위로 선택하는 비율입니다. 1보다 작은 값이면 샘플링이 수행되어 모델의 변동성을 줄일 수 있습니다.

- colsample_bytree: 0.9: 각 트리마다 사용되는 피처의 일부를 무작위로 선택하는 비율입니다. 이는 모델이 피처 간의 상관 관계를 다루는 데 도움을 줄 수 있습니다.

- objective: 'binary:logistic': 모델의 목적 함수로, 이진 분류 문제를 다루는 로지스틱 손실 함수를 사용하고 있음을 나타냅니다.

- eval_metric: 'auc': 모델의 평가 지표로, AUC (Area Under the Curve) 값을 사용하고 있음을 나타냅니다.

- d_train: XGBoost의 DMatrix 형식의 학습 데이터입니다.

- nrounds: 트리의 개수로, 반복 횟수를 설정합니다.

- evals=watchlist: 검증 데이터셋을 나타내며, 모델의 성능 평가를 위해 사용됩니다.

- early_stopping_rounds=100: 조기 중단을 수행하는 데 사용되는 값으로, 검증 데이터의 성능이 더 이상 향상되지 않을 때 훈련을 중지합니다.

- custom_metric=gini_xgb: 사용자 정의 메트릭 함수인 gini_xgb를 사용하여 모델을 평가합니다.

- maximize=True: 최대화 문제로 모델을 훈련하는 것을 나타냅니다.

- verbose_eval=10: 훈련 과정 중에 얼마나 자주 로그를 출력할지 설정합니다. 값이 클수록 더 자주 출력됩니다.

- 이렇게 설정된 매개변수와 옵션을 기반으로 XGBoost 모델이 훈련되며, 지정된 트리 개수 및 조기 중단 조건에 따라 최적의 모델이 선택될 것입니다.

In [ ]:
params = {'eta': 0.02, 'max_depth': 4,
          'subsample': 0.9,
          'colsample_bytree': 0.9, 
          'objective': 'binary:logistic',
          'eval_metric': 'auc',
          'silent': True}

In [28]:
xgb_model = xgb.train(params, d_train, nrounds, 
                      evals=watchlist, early_stopping_rounds=100,
                      custom_metric=gini_xgb, maximize=True, verbose_eval=10)

[13:04:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-auc:0.59974	train-gini:0.19795	valid-auc:0.59548	valid-gini:0.19166
[10]	train-auc:0.61712	train-gini:0.23422	valid-auc:0.61177	valid-gini:0.22349
[20]	train-auc:0.61992	train-gini:0.23979	valid-auc:0.61424	valid-gini:0.22849
[30]	train-auc:0.62237	train-gini:0.24474	valid-auc:0.61620	valid-gini:0.23239
[40]	train-auc:0.62316	train-gini:0.24632	valid-auc:0.61666	valid-gini:0.23331
[50]	train-auc:0.62358	train-gini:0.24715	valid-auc:0.61690	valid-gini:0.23380
[60]	train-auc:0.62379	train-gini:0.24758	valid-auc:0.61692	valid-gini:0.23383
[70]	train-auc:0.62403	train-gini:0.24805	valid-auc:0.61716	valid-gini:0.23433
[80]	train-auc:0.62456	train-gini:0.24912	valid-auc:0.61744	valid-gini:0.23489
[90]	train-auc:0.62551	train-gini:0.25102	valid-auc:0.61825	valid-gini:0.23651
[100]	train-auc

> xgb.train() 함수는 모델을 훈련하고 최적의 트리 개수를 찾는 과정을 수행
- XGBoost 모델을 훈련하는 코드입니다. 코드 실행이 완료되면, XGBoost 모델은 주어진 하이퍼파라미터와 훈련 데이터를 기반으로 학습됩니다. 

- params: XGBoost 모델의 하이퍼파라미터를 설정한 딕셔너리입니다.
- d_train: 훈련 데이터를 나타내는 DMatrix 객체입니다.
- nrounds: 훈련할 트리의 개수입니다.
- evals=watchlist: 훈련 및 검증 데이터의 성능을 모니터링하기 위한 목록입니다.
- early_stopping_rounds=100: 조기 중단을 위한 설정입니다.
- custom_metric=gini_xgb: 사용자 정의 메트릭 함수로서 모델의 성능을 평가하는데 사용됩니다.
- maximize=True: 성능 지표를 최대화하는 방향으로 모델을 훈련합니다.
- verbose_eval=100: 훈련 과정에서 출력을 제어하는 설정으로, 주어진 라운드마다 얼마나 자세한 정보를 출력할지 결정합니다.
- 위 코드를 실행하면, XGBoost 모델이 훈련되며 주어진 데이터와 하이퍼파라미터에 맞게 최적의 트리 개수를 찾게 됩니다. 훈련이 끝난 후, 트리 개수에 따른 검증 데이터의 성능 변화가 출력되며, 조기 중단이나 지정된 트리 개수까지 모델이 훈련됩니다.

In [29]:
params

{'eta': 0.02,
 'max_depth': 4,
 'subsample': 0.9,
 'colsample_bytree': 0.9,
 'objective': 'binary:logistic',
 'eval_metric': 'auc',
 'silent': True}

In [30]:
iteration_range = (xgb_model.best_ntree_limit, xgb_model.best_ntree_limit + 50)
sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        iteration_range=(xgb_model.best_ntree_limit, xgb_model.best_ntree_limit + 50)) / (2*kfold)

- XGBoost 모델을 사용하여 예측을 수행하고, 그 결과를 sub 데이터프레임의 'target' 열에 더하는 과정

- sub['target']: 'target' 열을 가리킵니다. 이 열에 예측 결과를 누적하려고 합니다.

- xgb_model.predict(...): xgb_model을 사용하여 테스트 데이터셋의 예측값을 계산합니다. xgb_model.predict 함수는 xgb.DMatrix 형식의 데이터를 입력으로 받아 모델의 예측값을 반환합니다.

- xgb.DMatrix(test[features].values): 테스트 데이터의 특성들을 XGBoost 모델에 맞는 형식인 xgb.DMatrix로 변환합니다. 여기서 features는 사용하려는 특성들을 나타내는 변수입니다.

- iteration_range=(xgb_model.best_ntree_limit, xgb_model.best_ntree_limit + 50): xgb_model의 best_ntree_limit 값에서 50개를 더한 범위 내에서 예측을 수행합니다. 이는 최상의 트리 개수를 기준으로 예측을 하되, 추가적으로 50개의 트리를 더 사용하여 예측합니다.

- / (2*kfold): 마지막으로, 예측 값을 (2 * kfold)로 나누어 더해줍니다. 이는 교차 검증을 통해 나눈 폴드 수의 두 배로 나누는 것으로, 앙상블된 예측 값을 균등하게 보정하는 역할을 합니다.

- 이 코드는 교차 검증된 XGBoost 모델의 예측 값을 앙상블하여 최종 예측 값을 생성하는 과정을 나타내고 있습니다

In [32]:
xgb_model.best_ntree_limit

860

In [33]:
sub   # 여기에는 1/4 의 값이 들어가 있는 것 bcz (2*kfold)

,id,target
0,0,0.124104
1,1,0.125182
2,2,0.124917
3,3,0.125141
4,4,0.125087
...,...,...
892811,1488022,0.125753
892812,1488023,0.125894
892813,1488024,0.126633
892814,1488025,0.127824


- xgboost 를 2개의 폴드로 나눠서 모델 2개로 0.5를 채운것
- lightgbm 을 2개의 폴드로 나눠서 모델 2개로 0.5를 채운것
- 1.0 전체 submission 나오고 그걸 제출 : 이것이 Stacking 이다...

In [34]:
gc.collect()

845

In [35]:
sub.head(2)

,id,target
0,0,0.124104
1,1,0.125182


In [43]:
# lgb
params = {'metric': 'auc', 'learning_rate' : 0.01,
          'max_depth':10, 'max_bin':10,  'objective': 'binary', 
          'feature_fraction': 0.8,'bagging_fraction':0.9,
          'bagging_freq':10,  'min_data': 500}

In [44]:
skf = StratifiedKFold(n_splits=kfold, shuffle= True, random_state=1)

In [45]:
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    break

In [46]:
X_train, X_eval = X[train_index], X[test_index]
y_train, y_eval = y[train_index], y[test_index]

In [54]:
lgb_model = lgb.train(params, lgb.Dataset(X_train, label=y_train), nrounds, 
                  lgb.Dataset(X_eval, label=y_eval), verbose_eval=True, 
                  feval=gini_lgb, early_stopping_rounds=100)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 10847, number of negative: 286759
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021904 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 200
[LightGBM] [Info] Number of data points in the train set: 297606, number of used features: 34
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036448 -> initscore=-3.274754
[LightGBM] [Info] Start training from score -3.274754


KeyboardInterrupt: 